In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Masking, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import matplotlib.pyplot as plt
import tensorflow as tf

In [29]:
df = pd.read_csv(r"C:\Users\KAVISH\Downloads\FidelFolio_df1.csv")

In [30]:
df = df.drop('Unnamed: 0', axis=1)

In [31]:
for col in df.columns:
    if col not in ['Company', 'Year']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [32]:
df['target_lag1'] = df.groupby('Company')['Target 1'].shift(1)
#df['Target1_lag1'] = df['target_lag1'].fillna(-999)

In [33]:
feature_cols = [col for col in df.columns if col not in ['Company', 'Year', 'Target 1']]

In [34]:
df[feature_cols] = df[feature_cols].fillna(-999)

**MODEL**

In [35]:
SEQ_LEN = 3  # Increased sequence length
MASK_VALUE = -999.
BATCH_SIZE = 64  # Adjusted batch size
EPOCHS = 30  # Increased epochs with early stopping
INIT_LR = 0.0005  # Lower initial learning rate


In [36]:
df = df.sort_values(by=['Company', 'Year'])

In [40]:
df

,Unnamed: 0.1,Year,Company,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,...,sector_5,sector_6,sector_7,sector_8,sector_9,sector_10,sector_11,sector_12,Target 1,target_lag1
0,0,2020,360 ONE,-0.025730,-0.386291,-0.421173,-0.062958,-0.013804,-0.025585,0.007415,...,0,1,0,0,0,0,0,0,-51.37,-999.00
1,1,2021,360 ONE,-0.030814,-0.266381,-0.070393,-0.067283,-0.001258,-0.021895,-0.005882,...,0,1,0,0,0,0,0,0,18.58,-51.37
2,2,2022,360 ONE,-0.116921,0.065501,0.351499,-0.142967,0.535244,-0.075770,-0.003155,...,0,1,0,0,0,0,0,0,5.24,18.58
3,3,2023,360 ONE,-0.116852,0.060860,0.450697,-0.358591,0.373997,-0.025904,-0.019264,...,0,1,0,0,0,0,0,0,29.26,5.24
4,4,2024,360 ONE,-0.116700,0.148279,0.634154,-0.358591,-0.056433,-0.016513,-0.012616,...,0,1,0,0,0,0,0,0,NaN,29.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24231,24745,2023,eClerx Services,-0.271570,0.902688,0.634059,-0.047205,0.025891,-0.021611,0.036971,...,0,0,0,0,1,0,0,0,53.84,-19.40
24232,24746,2024,eClerx Services,-0.271530,0.618503,0.401177,-0.033805,0.031491,0.256976,0.037231,...,0,0,0,0,1,0,0,0,NaN,53.84
24233,24747,2014,eDynamics Solut.,-0.267946,-1.316809,-1.102025,1.951386,0.550050,-0.056749,0.246826,...,0,0,0,0,1,0,0,0,-122.38,-999.00
24234,24748,2023,eMudhra,-0.271326,0.152514,0.205158,-0.027223,0.048863,-0.063738,0.029430,...,0,0,0,0,1,0,0,0,242.39,-999.00


In [37]:
X_train = df[[col for col in feature_cols if col in df.columns]].values

#sector_train = df1['SectorCode_encoded'].values

y_train = df['Target 1'].values

In [38]:
X_train

array([[ 0.00000000e+00, -2.57304995e-02, -3.86291382e-01, ...,
         0.00000000e+00,  0.00000000e+00, -9.99000000e+02],
       [ 1.00000000e+00, -3.08139886e-02, -2.66380621e-01, ...,
         0.00000000e+00,  0.00000000e+00, -5.13700000e+01],
       [ 2.00000000e+00, -1.16921032e-01,  6.55014192e-02, ...,
         0.00000000e+00,  0.00000000e+00,  1.85800000e+01],
       ...,
       [ 2.47470000e+04, -2.67945550e-01, -1.31680883e+00, ...,
         0.00000000e+00,  0.00000000e+00, -9.99000000e+02],
       [ 2.47480000e+04, -2.71325991e-01,  1.52513980e-01, ...,
         0.00000000e+00,  0.00000000e+00, -9.99000000e+02],
       [ 2.47490000e+04, -2.71896185e-01, -2.39636365e-01, ...,
         0.00000000e+00,  0.00000000e+00,  2.42390000e+02]])

In [54]:

train_df = df[df['Year'] < 2024].copy()
test_df = df[df['Year'] == 2024].copy()


#train_df = train_df.dropna(subset=[' Target 1 '])

# Fill NaNs in features (you can also choose to drop or impute)
X_columns = [col for col in train_df.columns if col not in ['Company', 'Year', 'Target 1','Target 2', 'Target 3']]
train_df[X_columns] = train_df[X_columns].fillna(-999)
test_df[X_columns] = test_df[X_columns].fillna(-999)
for col in X_columns:
    train_df[col] = train_df[col].replace(-999, np.nan)
    train_df[col] = train_df.groupby('Company')[col].ffill()
    train_df[col] = train_df[col].fillna(2)
latest_train = train_df.groupby('Company').tail(1)
test_combined = pd.concat([latest_train, test_df], axis=0)
test_combined = test_combined.sort_values(by=['Company', 'Year'])

# Apply same ffill logic to test_combined
for col in X_columns:
    test_combined[col] = test_combined[col].replace(-999, np.nan)
    test_combined[col] = test_combined.groupby('Company')[col].ffill()
    test_combined[col] = test_combined[col].fillna(2)

# Extract test_df again
test_df = test_combined[test_combined['Year'] == 2024]


In [55]:
X_train = train_df[X_columns]
y_train = train_df['Target 1'].values

X_test = test_df[X_columns]

# Build the MLP model
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=-999.0, input_shape=(X_train.shape[1],)),
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(1024, activation='sigmoid'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Regression output
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Predict for 2024
predictions_2024 = model.predict(X_test)

# Add predictions to test_df
test_df['predicted_target1'] = predictions_2024

# Save the predictions
test_df[['Company', 'Year', 'predicted_target1']].to_csv("2024_predictions.csv", index=False)

Epoch 1/50
575/575 [==============================] - 7s 9ms/step - loss: 12054.7920 - mae: 57.3351 - val_loss: 36783.1484 - val_mae: 65.1570
Epoch 2/50
575/575 [==============================] - 5s 9ms/step - loss: 12047.4082 - mae: 57.3274 - val_loss: 36759.8555 - val_mae: 61.0241
Epoch 3/50
575/575 [==============================] - 5s 8ms/step - loss: 12024.7461 - mae: 57.4903 - val_loss: 36740.2227 - val_mae: 61.9531
Epoch 4/50
575/575 [==============================] - 5s 9ms/step - loss: 12036.7051 - mae: 57.5432 - val_loss: 36744.5391 - val_mae: 61.6486
Epoch 5/50
575/575 [==============================] - 5s 8ms/step - loss: 12028.6719 - mae: 57.5131 - val_loss: 36778.8047 - val_mae: 60.5631
Epoch 6/50
575/575 [==============================] - 5s 8ms/step - loss: 12026.8057 - mae: 57.4461 - val_loss: 36786.7930 - val_mae: 60.4080
Epoch 7/50
575/575 [==============================] - 5s 9ms/step - loss: 12022.6592 - mae: 57.5406 - val_loss: 36781.0234 - val_mae: 60.5176
Epoch 

C:\Users\KAVISH\AppData\Local\Temp\ipykernel_18680\833319284.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predicted_target1'] = predictions_2024


In [56]:
test_df

,Unnamed: 0.1,Year,Company,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,...,sector_6,sector_7,sector_8,sector_9,sector_10,sector_11,sector_12,Target 1,target_lag1,predicted_target1
4,4,2024,360 ONE,-0.116700,0.148279,0.634154,-0.358591,-0.056433,-0.016513,-0.012616,...,1,0,0,0,0,0,0,NaN,29.26,6.267790
30,30,2024,3M India,-0.136963,1.692996,0.768591,0.095645,0.069494,0.001628,0.029870,...,0,0,1,0,0,0,0,NaN,7.21,4.284550
53,53,2024,5Paisa Capital,-0.117415,0.064728,-0.180348,-0.067489,-0.029228,-0.030673,-0.017986,...,1,0,0,0,0,0,0,NaN,52.89,12.968574
76,76,2024,63 Moons Tech.,-0.547462,-0.920315,-0.725786,-0.054204,0.040962,-0.073312,0.023709,...,0,0,0,1,0,0,0,NaN,89.70,13.274706
102,102,2024,A B B,-0.136963,0.965005,0.398278,-1.715893,0.095783,-0.040615,0.028227,...,0,0,1,0,0,0,0,NaN,45.58,3.686287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24167,24681,2024,Zuari Industries,-0.164119,-0.969101,-0.874169,-0.060304,0.039537,0.003676,0.000059,...,0,0,0,0,1,0,0,NaN,-48.55,16.912262
24193,24707,2024,Zydus Lifesci.,-0.035894,0.243720,0.046461,0.035244,0.015981,0.029722,0.026326,...,0,1,0,0,0,0,0,NaN,74.71,16.912262
24209,24723,2024,Zydus Wellness,-0.251292,-0.810299,-0.525901,-0.049461,0.076428,-0.003015,-0.000811,...,0,0,0,0,0,0,0,NaN,-33.71,16.912262
24232,24746,2024,eClerx Services,-0.271530,0.618503,0.401177,-0.033805,0.031491,0.256976,0.037231,...,0,0,0,1,0,0,0,NaN,53.84,16.912262


In [57]:
test_df.to_csv('test_data_processed1.csv', index=False)

In [31]:
df_out = df[df["Year"] == 2024].copy()
df_out = df_out.drop(columns=['Company', 'Year'], errors='ignore')

# Ensure all training-time features are present
for col in feature_cols:
    if col not in df_out.columns:
        df_out[col] = 0  # Fill missing columns with zeros

# Align columns and fill missing values
df_out = df_out[feature_cols].fillna(0)

# --- STEP 2: Scale the features ---
X_out_scaled = scaler_X.transform(df_out)

# --- STEP 3: Create rolling sequences ---
X_out_seq = []
for i in range(len(X_out_scaled) - SEQ_LEN):
    seq = X_out_scaled[i:i + SEQ_LEN]
    X_out_seq.append(seq)

# Convert to NumPy array
X_out_seq = np.array(X_out_seq)

# --- STEP 4: Predict row-by-row independently ---
predictions = []
for seq in X_out_seq:
    seq = np.expand_dims(seq, axis=0)  # shape (1, SEQ_LEN, num_features)
    y_pred_scaled = model.predict(seq, verbose=0).flatten()[0]
    y_pred = scaler_y.inverse_transform([[y_pred_scaled]])[0][0]
    predictions.append(y_pred)

# --- STEP 5: Save predictions ---
df_pred = df_out.iloc[SEQ_LEN:].copy()  # align with predictions (SEQ_LEN lost)
df_pred['Predicted Target 1'] = predictions

# --- STEP 6: Output the result ---
df_pred[['Predicted Target 1']]

,Predicted Target 1
23508,17.024160
23509,16.988226
23510,16.980681
23511,16.987201
23512,16.967608
...,...
24746,16.939202
24747,16.930475
24748,16.935082
24749,16.938628
